<b style="font-family: Times New Roman; font-size:1.2em;color:black;"> I choose COVID dataset for this project.From this data set I tried to calulate stringency_index of Poland and predict it with new data.Actual stringency_index is derived from various factor of lockdown implemented during the COVID period of year 2020 . This is a composite measure based on nine response indicators including school closures, workplace closures, and travel bans, rescaled to a value from 0 to
100 (100 = strictest).Our dataset already has stringency_index derived based upon the mentioned reponses.Now as part of this ML project we train our model to derive stringency_index from other available data in dataset.</b>


In [6]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [7]:
from azureml.core import Workspace, Dataset
ws = Workspace.from_config()

<b style="font-family: Times New Roman; font-size:1.2em;color:black;">Upload dataset from AML Dataset and verify the loaded dataset.</b>

In [9]:
aml_dataset = ws.datasets['COVID_PL1']

# Use Pandas DataFrame just to sneak peak some data and schema
full_df = aml_dataset.to_pandas_dataframe()
# .to_pandas_dataframe().dropna()
full_df.head(5)
full_df.describe()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,median_age,aged_65_older,aged_70_older,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index
count,3.030000e+02,303.000000,214.000000,295.000000,295.000000,298.000000,143.000000,303.000000,298.000000,295.000000,...,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02
mean,2.303057e+05,4273.524752,1415.412565,4821.471186,96.793220,92.207570,1338.339350,112.917020,112.311221,127.395081,...,4.180000e+01,1.676300e+01,1.020200e+01,2.273310e+02,5.910000e+00,2.330000e+01,3.310000e+01,6.620000e+00,7.873000e+01,8.650000e-01
std,3.793011e+05,7139.003549,4193.718142,7271.570915,169.098454,148.561523,4574.412658,188.629956,183.771719,192.132717,...,1.708124e-13,6.049604e-14,7.650970e-14,8.825305e-13,2.935837e-14,1.174335e-13,1.850467e-13,5.159957e-14,3.558591e-13,3.447385e-15
min,1.000000e+00,0.000000,2.286000,1.000000,0.000000,0.000000,0.026000,0.000000,0.060000,0.026000,...,4.180000e+01,1.676300e+01,1.020200e+01,2.273310e+02,5.910000e+00,2.330000e+01,3.310000e+01,6.620000e+00,7.873000e+01,8.650000e-01
25%,1.907650e+04,313.000000,304.642750,1001.500000,9.000000,10.143000,142.417000,8.270000,8.847750,26.462000,...,4.180000e+01,1.676300e+01,1.020200e+01,2.273310e+02,5.910000e+00,2.330000e+01,3.310000e+01,6.620000e+00,7.873000e+01,8.650000e-01
50%,4.689400e+04,548.000000,371.143000,1774.000000,16.000000,14.571500,465.431000,14.480000,12.986500,46.873000,...,4.180000e+01,1.676300e+01,1.020200e+01,2.273310e+02,5.910000e+00,2.330000e+01,3.310000e+01,6.620000e+00,7.873000e+01,8.650000e-01
75%,1.624190e+05,4887.000000,591.499750,3593.500000,75.000000,80.678750,823.468500,129.126500,184.430500,94.949000,...,4.180000e+01,1.676300e+01,1.020200e+01,2.273310e+02,5.910000e+00,2.330000e+01,3.310000e+01,6.620000e+00,7.873000e+01,8.650000e-01
max,1.294878e+06,32733.000000,25611.000000,28554.000000,674.000000,505.286000,33234.130000,864.886000,676.705000,754.467000,...,4.180000e+01,1.676300e+01,1.020200e+01,2.273310e+02,5.910000e+00,2.330000e+01,3.310000e+01,6.620000e+00,7.873000e+01,8.650000e-01


<p style="font-family: Times New Roman; font-size:1.2em;color:black;">Data Cleaning: Our dataset contains many fields which are not required for calculating stringency_index. Hence we remove those unnecessary fields , this operation will help to calculate our prediction fast and higher accuracy.Data cleansing activity is one of the primary task before creating any ML model.  </p>


In [12]:
aml_dataset = aml_dataset.drop_columns(['new_cases_smoothed','new_deaths_smoothed','new_cases_smoothed_per_million',
                                        'new_deaths_smoothed_per_million','hosp_patients','hosp_patients_per_million','total_tests_per_thousand','new_tests_per_thousand',
                                       'new_tests_smoothed','new_tests_smoothed_per_thousand','tests_per_case','total_vaccinations','new_vaccinations',
                                       'new_vaccinations_smoothed','total_vaccinations_per_hundred','new_vaccinations_smoothed_per_million',
                                       'median_age','aged_65_older','aged_70_older','gdp_per_capita','cardiovasc_death_rate','diabetes_prevalence',
                                        'female_smokers','male_smokers','hospital_beds_per_thousand','life_expectancy','human_development_index'])

full_df = aml_dataset.to_pandas_dataframe()
full_df.describe()

,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,reproduction_rate,total_tests,new_tests,positive_rate,stringency_index,population,population_density
count,3.030000e+02,303.000000,295.000000,295.000000,143.000000,303.000000,295.000000,295.000000,292.000000,2.470000e+02,245.000000,241.000000,303.000000,303.0,3.030000e+02
mean,2.303057e+05,4273.524752,4821.471186,96.793220,1338.339350,112.917020,127.395081,2.557512,1.169212,2.993179e+06,27071.061224,0.131963,56.601320,37846605.0,1.240270e+02
std,3.793011e+05,7139.003549,7271.570915,169.098454,4574.412658,188.629956,192.132717,4.468007,0.322892,1.992320e+06,14471.272568,0.155023,20.899427,0.0,5.124371e-13
min,1.000000e+00,0.000000,1.000000,0.000000,0.026000,0.000000,0.026000,0.000000,0.750000,2.817870e+05,7304.000000,0.013000,11.110000,37846605.0,1.240270e+02
25%,1.907650e+04,313.000000,1001.500000,9.000000,142.417000,8.270000,26.462000,0.238000,0.960000,1.334627e+06,18014.000000,0.022000,39.810000,37846605.0,1.240270e+02
50%,4.689400e+04,548.000000,1774.000000,16.000000,465.431000,14.480000,46.873000,0.423000,1.050000,2.587559e+06,21579.000000,0.032000,53.700000,37846605.0,1.240270e+02
75%,1.624190e+05,4887.000000,3593.500000,75.000000,823.468500,129.126500,94.949000,1.982000,1.300000,4.508448e+06,33020.000000,0.299000,75.000000,37846605.0,1.240270e+02
max,1.294878e+06,32733.000000,28554.000000,674.000000,33234.130000,864.886000,754.467000,17.809000,2.330000,6.941502e+06,75559.000000,0.503000,83.330000,37846605.0,1.240270e+02


<b style="font-family: Times New Roman; font-size:1.2em;color:black;"> This query will show the stringency_index of Poland from the begning.
Here we show first 25 days. </b>

In [59]:

columns = ['date','stringency_index']
aml_dataset = ws.datasets['COVID_PL1'].keep_columns(columns, validate=False)
full_df = aml_dataset.to_pandas_dataframe()
full_df.head(25)

,date,stringency_index
0,2020-03-04,11.11
1,2020-03-05,11.11
2,2020-03-06,11.11
3,2020-03-07,11.11
4,2020-03-08,11.11
5,2020-03-09,13.89
6,2020-03-10,25.00
7,2020-03-11,25.00
8,2020-03-12,41.67
9,2020-03-13,41.67


<b style="font-family: Times New Roman; font-size:1.2em;color:black;">Importing Azure libraries for AmlCompute and listing the available ComputeTarget list availabe in workspace.</b>

In [21]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

ComputeTarget.list(ws)

[AmlCompute(workspace=Workspace.create(name='AML', subscription_id='1e714b1d-ff6c-4b32-841d-981d38f7622c', resource_group='pws'), name=pws1, id=/subscriptions/1e714b1d-ff6c-4b32-841d-981d38f7622c/resourceGroups/pws/providers/Microsoft.MachineLearningServices/workspaces/AML/computes/pws1, type=AmlCompute, provisioning_state=Succeeded, location=westus2, tags=None),
 {
   "id": "/subscriptions/1e714b1d-ff6c-4b32-841d-981d38f7622c/resourceGroups/pws/providers/Microsoft.MachineLearningServices/workspaces/AML/computes/pws2",
   "name": "pws2",
   "location": "westus2",
   "tags": null,
   "properties": {
     "description": null,
     "computeType": "ComputeInstance",
     "computeLocation": "westus2",
     "resourceId": null,
     "provisioningErrors": null,
     "provisioningState": "Succeeded",
     "properties": {
       "vmSize": "STANDARD_DS11_V2",
       "applications": [
         {
           "displayName": "Jupyter",
           "endpointUri": "https://pws2.westus2.instances.azureml.

<b style="font-family: Times New Roman; font-size:1.2em;color:black;">Checking the available Compute targets</b>

In [25]:
cts = ws.compute_targets
print (cts)

{'pws1': AmlCompute(workspace=Workspace.create(name='AML', subscription_id='1e714b1d-ff6c-4b32-841d-981d38f7622c', resource_group='pws'), name=pws1, id=/subscriptions/1e714b1d-ff6c-4b32-841d-981d38f7622c/resourceGroups/pws/providers/Microsoft.MachineLearningServices/workspaces/AML/computes/pws1, type=AmlCompute, provisioning_state=Succeeded, location=westus2, tags=None), 'pws2': {
  "id": "/subscriptions/1e714b1d-ff6c-4b32-841d-981d38f7622c/resourceGroups/pws/providers/Microsoft.MachineLearningServices/workspaces/AML/computes/pws2",
  "name": "pws2",
  "location": "westus2",
  "tags": null,
  "properties": {
    "description": null,
    "computeType": "ComputeInstance",
    "computeLocation": "westus2",
    "resourceId": null,
    "provisioningErrors": null,
    "provisioningState": "Succeeded",
    "properties": {
      "vmSize": "STANDARD_DS11_V2",
      "applications": [
        {
          "displayName": "Jupyter",
          "endpointUri": "https://pws2.westus2.instances.azureml.ms

In [27]:
amlcompute_cluster_name = "pws1"

cts = ws.compute_targets

aml_remote_compute = cts[amlcompute_cluster_name]

<b style="font-family: Times New Roman; font-size:1.2em;color:black;">Import AutoML library</b> 

In [29]:
from azureml.train import automl

In [30]:
from azureml.train.automl import AutoMLConfig

<b style="font-family: Times New Roman; font-size:1.2em;color:black;">List and select primary metric to drive our AutoML regression problem</b>

In [31]:
automl.utilities.get_primary_metrics('regression')

['normalized_root_mean_squared_error',
 'spearman_correlation',
 'normalized_mean_absolute_error',
 'r2_score']

<b style="font-family: Times New Roman; font-size:1.2em;color:black;">Split original AML Tabular Dataset in two test/train AML Tabular Datasets (using AML DS function) </b>

In [32]:
train_dataset, test_dataset = aml_dataset.random_split(0.9, seed=1)

# Use Pandas DF only to check the data
train_dataset_df = train_dataset.to_pandas_dataframe()
test_dataset_df = test_dataset.to_pandas_dataframe()

print(train_dataset_df.describe())
# print(test_dataset_df.describe())

       stringency_index
count        272.000000
mean          56.015846
std           21.052659
min           11.110000
25%           39.810000
50%           53.700000
75%           75.000000
max           83.330000


<b style="font-family: Times New Roman; font-size:1.2em;color:black;">Configuration setting for running AutoML experiment. </b>

In [34]:
import logging
import os

from azureml.train.automl import AutoMLConfig

project_folder = './automl'
os.makedirs(project_folder, exist_ok=True)

automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='regression',
                             primary_metric='normalized_root_mean_squared_error',
                             experiment_timeout_minutes=15,                            
                             training_data=train_dataset,
                             label_column_name="stringency_index",
                             n_cross_validations=5,
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             path=project_folder
                             )

<b style="font-family: Times New Roman; font-size:1.2em;color:black;">Experiment with multiple child runs.The result shows best model with time duration and metric.</b>

In [35]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "regression-automl-remote-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=ws, name=experiment_name)

import time
start_time = time.time()
            
run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))

regression-automl-remote-01-28-2021-20
Running on remote.
No run_configuration provided, running on pws1 with default configuration
Running on remote compute: pws1
Parent Run ID: AutoML_65ed0f9e-d0bb-41c6-9dc7-e5a743c0cdf0

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn

<b style="font-family: Times New Roman; font-size:1.2em;color:black;">Explore results with Widget.</b>

In [66]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

<b style="font-family: Times New Roman; font-size:1.2em;color:black;">Show the best fitted model from all models. </b>

In [36]:
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.3, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0.post1, current version:1.19.0


Run(Experiment: regression-automl-remote-01-28-2021-20,
Id: AutoML_65ed0f9e-d0bb-41c6-9dc7-e5a743c0cdf0_15,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                       

<b style="font-family: Times New Roman; font-size:1.2em;color:black;">To make the prediction we remove label y column from dataset and save it with different name. Later this dataset will be compared with other dataset for accuracy. </b>

In [37]:
import pandas as pd

#Remove Label/y column
if 'stringency_index' in test_dataset_df.columns:
    y_test_df = test_dataset_df.pop('stringency_index')

x_test_df = test_dataset_df

In [38]:
y_predictions = fitted_model.predict(x_test_df)

print('10 predictions: ')
print(y_predictions[:10])

10 predictions: 
[57.41106424 57.41106424 83.30539599 83.31134038 83.30527501 78.5418556
 53.66758739 53.66770837 53.66770837 50.92039527]


In [50]:
y_predictions.shape

(31,)

In [56]:
print(x_test_df)

         date
0  2020-03-26
1  2020-03-29
2  2020-04-10
3  2020-04-29
4  2020-05-06
5  2020-05-26
6  2020-06-03
7  2020-06-11
8  2020-06-12
9  2020-06-28
10 2020-07-13
11 2020-08-20
12 2020-09-15
13 2020-09-21
14 2020-09-25
15 2020-10-06
16 2020-10-16
17 2020-10-26
18 2020-10-29
19 2020-11-02
20 2020-11-04
21 2020-11-07
22 2020-11-10
23 2020-11-19
24 2020-11-25
25 2020-11-26
26 2020-11-28
27 2020-12-01
28 2020-12-13
29 2020-12-18
30 2020-12-23


<b style="font-family: Times New Roman; font-size:1.2em;color:black;">Make the actual Predictions</b>

In [57]:
print(y_test_df)

0    57.41
1    57.41
2    83.33
3    83.33
4    83.33
5    79.63
6    53.70
7    53.70
8    53.70
9    50.93
10   39.81
11   39.81
12   36.11
13   23.15
14   23.15
15   23.15
16   37.04
17   71.30
18   71.30
19   71.30
20   71.30
21   75.00
22   75.00
23   75.00
24   75.00
25   75.00
26   75.00
27   75.00
28   75.00
29   75.00
30   75.00
Name: stringency_index, dtype: float64


In [58]:
from sklearn.metrics import accuracy_score

print('Accuracy:')
accuracy_score(y_test_df, y_predictions)

Accuracy:


ValueError: continuous is not supported

<b style="font-family: Times New Roman; font-size:1.2em;color:black;"> While comparing accuracy I got error as shown above.</b>